# Keyword spam model MVP

## Setup

You'll need to download en_core_web_sm for this task

In [ ]:
pip install -r requirements.txt

In [1]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7 MB 7.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/alicejacques/.virtualenvs/ml-coding-test-check/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import nltk
nltk.download('punkt')

In [2]:
from itertools import chain
import os
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
import spacy
import truecase
import xgboost as xgb
import unidecode

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
nlp = spacy.load("en_core_web_sm")

## Read data

In [3]:
cwd = os.getcwd()

train = pd.read_csv(os.path.join(cwd, "data/train_set.tsv"), sep="\t")[
    ["product_id", "description", "label"]
]
test = pd.read_csv(os.path.join(cwd, "data/test_set.tsv"), sep="\t")[
    ["product_id", "description", "label"]
]
df = pd.concat([train, test], axis=0)

df = df.reset_index(drop=True)
df[:2]

,product_id,description,label
0,180202844,"super cute high waisted blue jeans. recommended for shorter / petite girls, i'm 5'1 and it fits me perfect. (example shown in last pic) these are more of jeggings than jeans and have a polyester material inside. only flaw: the zipper goes down by itself sometimes branded for exposure size 0 / waist 23",0
1,178987418,Jordan 5 P51 Camo Size 9 Good condition Soles have yellowing No box or lacelocks $160 Travis Supreme concepts Jordan 1 3 5 6 7 11 12 Nike ovo Kanye yeezy boost 350 shadow royal bred shattered cement top 3 black toe infrared raptor gamma space jam air max vapormax flyknit Travis Scott kaws off white atmos 95 97 98 silver gold bullet protro Kobe fieg kith Levi’s undefeated Palace tinker stash SB dunk stone island Foamposite plus Acronym VF Wotherspoon 270 SW LeBron Kyrie Pippen Metallic Penny Orlando Dunkman Eggplant Chrome All Star Pearl Camo JTH powder olive protro presto off white butter He got game cdg bogo Paris saint Germain Court purple pine green blazer union diamond Dunk cdg staple pigeon Diamond resale bape Zoom fly concord union Purple lobster concepts 997 converse homage,1


## Useful feature engineering functions

In [4]:
def proportion_of_stopwords(description):
    tokens = description.split(" ")
    num_stopwords = len(
        [word for word in tokens if word.lower() in nlp.Defaults.stop_words]
    )
    return float(num_stopwords) / float(len(tokens)) if len(tokens) else 0.0


def average_length_of_word(description):
    tokens = description.split(" ")
    return np.mean([len(word) for word in tokens]) if len(tokens) else 0.0


def proportion_of_numbers(description):
    tokens = description.split(" ")
    num_digits = len([word for word in tokens if word.isdigit()])
    return float(num_digits) / float(len(tokens)) if len(tokens) else 0.0


def normalise_nonascii_chars(input_str):
    return unidecode.unidecode(input_str)


def replace_special_chars(main_string):
    return re.sub("[,;@#!\?\+\*\n\-: /]", " ", main_string)


def keep_alphanumeric_chars(string_input):
    return re.sub("[^A-Za-z0-9& ]", "", string_input)


def remove_spaces(string_input):
    return " ".join(string_input.split())


def lemmatize(string_input):
    token_object = nlp(string_input)
    lemmas_list = [
        word.lemma_ if word.lemma_ != "-PRON-" else word.text for word in token_object
    ]
    return " ".join(lemmas_list)


def clean_description(input_str):
    input_str = replace_special_chars(input_str.lower())
    input_str = normalise_nonascii_chars(input_str)
    input_str = keep_alphanumeric_chars(input_str)
    input_str = lemmatize(input_str)
    input_str = remove_spaces(input_str)
    return input_str


## Start building features

In [5]:
df['description'] = df['description'].apply(clean_description)

for i in ['proportion_of_stopwords', 'average_length_of_word', 'proportion_of_numbers']:
    df[i] = df['description'].apply(eval(i))
    
df[:1]    

,product_id,description,label,proportion_of_stopwords,average_length_of_word,proportion_of_numbers
0,180202844,super cute high waiste blue jean recommend for short petite girl I m 51 and it fit I perfect example show in last pic these be more of jegging than jean and have a polyester material inside only flaw the zipper go down by itself sometimes brand for exposure size 0 waist 23,0,0.45283,4.169811,0.056604


## Have a go at identifying brands, and nouns likely to be spammed

In [6]:
def drop_digits(s):
    return "".join([i for i in s if not i.isdigit()])


df["description"] = df["description"].apply(drop_digits)
# .ents only stands a chance with capitalized words
df["description_truecase"] = df["description"].apply(truecase.get_true_case)
df["description_nlp"] = df["description_truecase"].apply(nlp)

df["named_entities"] = ""
for i, description_nlp in df["description_nlp"].iteritems():
    named_entities_sets = description_nlp.ents
    named_entities = list(set(chain(*named_entities_sets)))
    df["named_entities"].iloc[i] = " ".join(j.text for j in named_entities)

/Users/alicejacques/.virtualenvs/ml-coding-test/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Vectorise the top 500 named entities

In [7]:
vectorizer_named_entities = TfidfVectorizer(
    stop_words=nlp.Defaults.stop_words, max_features=500
)
df_tfidf_named_entities = pd.DataFrame(
    vectorizer_named_entities.fit_transform(df["named_entities"]).todense(),
    columns=vectorizer_named_entities.get_feature_names(),
)
df_tfidf_named_entities.columns = [
    f"{i}_named_entities" for i in df_tfidf_named_entities.columns
]
df = pd.concat([df, df_tfidf_named_entities], axis=1)
df[:10]

/Users/alicejacques/.virtualenvs/ml-coding-test/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,product_id,description,label,proportion_of_stopwords,average_length_of_word,proportion_of_numbers,description_truecase,description_nlp,named_entities,abercrombie_named_entities,adida_named_entities,adidas_named_entities,air_named_entities,airmax_named_entities,aj_named_entities,aje_named_entities,alexander_named_entities,alice_named_entities,alt_named_entities,amara_named_entities,american_named_entities,angel_named_entities,angele_named_entities,anime_named_entities,approx_named_entities,argyle_named_entities,ariana_named_entities,ashley_named_entities,ashlyn_named_entities,asos_named_entities,australia_named_entities,autumn_named_entities,babe_named_entities,babydoll_named_entities,babygirl_named_entities,backless_named_entities,baddie_named_entities,baguette_named_entities,baker_named_entities,balenciaga_named_entities,bandana_named_entities,bandeau_named_entities,bape_named_entities,barbie_named_entities,bardot_named_entities,barkingmadz_named_entities,bay_named_entities,bdg_named_entities,beach_named_entities,bead_named_entities,beanie_named_entities,bebe_named_entities,bec_named_entities,bella_named_entities,belle_named_entities,bershka_named_entities,betty_named_entities,bikini_named_entities,billie_named_entities,black_named_entities,blair_named_entities,bleach_named_entities,blogger_named_entities,blue_named_entities,bodycon_named_entities,bodysuit_named_entities,bohemian_named_entities,boho_named_entities,boob_named_entities,boohoo_named_entities,boop_named_entities,boujee_named_entities,boyfriend_named_entities,bra_named_entities,braid_named_entities,bralet_named_entities,bralette_named_entities,brandy_named_entities,brandymelville_named_entities,brat_named_entities,bratz_named_entities,britney_named_entities,broderie_named_entities,brown_named_entities,bts_named_entities,bubblegum_named_entities,burberry_named_entities,burgundy_named_entities,bustier_named_entities,busty_named_entities,california_named_entities,calvin_named_entities,cami_named_entities,camisole_named_entities,camo_named_entities,canada_named_entities,cara_named_entities,carhartt_named_entities,carla_named_entities,carlson_named_entities,caroline_named_entities,cb_named_entities,cdg_named_entities,chamberlain_named_entities,chanel_named_entities,cherry_named_entities,cherub_named_entities,chinese_named_entities,choker_named_entities,christy_named_entities,clark_named_entities,clasp_named_entities,club_named_entities,clueless_named_entities,co_named_entities,comfy_named_entities,coord_named_entities,corona_named_entities,corset_named_entities,cosplay_named_entities,cottagecore_named_entities,couture_named_entities,cowgirl_named_entities,crewneck_named_entities,croptop_named_entities,cutie_named_entities,cyber_named_entities,daisy_named_entities,davidson_named_entities,day_named_entities,de_named_entities,deadstock_named_entities,decathlon_named_entities,delia_named_entities,depop_named_entities,depopfamous_named_entities,depopseller_named_entities,depopusa_named_entities,diamante_named_entities,diamond_named_entities,dickie_named_entities,dior_named_entities,disco_named_entities,disney_named_entities,ditsy_named_entities,diy_named_entities,dm_named_entities,doc_named_entities,dolce_named_entities,dollskill_named_entities,dot_named_entities,dr_named_entities,dragon_named_entities,drmarten_named_entities,dupe_named_entities,dutch_named_entities,dye_named_entities,eagle_named_entities,earre_named_entities,earring_named_entities,eboy_named_entities,egirl_named_entities,eilish_named_entities,elasticate_named_entities,elizabeth_named_entities,ellish_named_entities,embroider_named_entities,emma_named_entities,emo_named_entities,erica_named_entities,evening_named_entities,express_named_entities,fall_named_entities,faux_named_entities,fendi_named_entities,fenty_named_entities,fila_named_entities,fishnet_named_entities,flea_named_entities,fleece_named_entities,flipflop_named_entities,flowery_named_entities,flowy_named_entities,force_named_entities,frankie_nam

## Vectorise the top 500 description words

In [8]:
vectorizer_description = TfidfVectorizer(
    stop_words=nlp.Defaults.stop_words, max_features=500
)
df_tfidf_description = pd.DataFrame(
    vectorizer_description.fit_transform(df["description"]).todense(),
    columns=vectorizer_description.get_feature_names(),
)
df_tfidf_description.columns = [
    f"{i}_description" for i in df_tfidf_description.columns
]
df = pd.concat([df, df_tfidf_description], axis=1)
df[:1]

/Users/alicejacques/.virtualenvs/ml-coding-test/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,product_id,description,label,proportion_of_stopwords,average_length_of_word,proportion_of_numbers,description_truecase,description_nlp,named_entities,abercrombie_named_entities,adida_named_entities,adidas_named_entities,air_named_entities,airmax_named_entities,aj_named_entities,aje_named_entities,alexander_named_entities,alice_named_entities,alt_named_entities,amara_named_entities,american_named_entities,angel_named_entities,angele_named_entities,anime_named_entities,approx_named_entities,argyle_named_entities,ariana_named_entities,ashley_named_entities,ashlyn_named_entities,asos_named_entities,australia_named_entities,autumn_named_entities,babe_named_entities,babydoll_named_entities,babygirl_named_entities,backless_named_entities,baddie_named_entities,baguette_named_entities,baker_named_entities,balenciaga_named_entities,bandana_named_entities,bandeau_named_entities,bape_named_entities,barbie_named_entities,bardot_named_entities,barkingmadz_named_entities,bay_named_entities,bdg_named_entities,beach_named_entities,bead_named_entities,beanie_named_entities,bebe_named_entities,bec_named_entities,bella_named_entities,belle_named_entities,bershka_named_entities,betty_named_entities,bikini_named_entities,billie_named_entities,black_named_entities,blair_named_entities,bleach_named_entities,blogger_named_entities,blue_named_entities,bodycon_named_entities,bodysuit_named_entities,bohemian_named_entities,boho_named_entities,boob_named_entities,boohoo_named_entities,boop_named_entities,boujee_named_entities,boyfriend_named_entities,bra_named_entities,braid_named_entities,bralet_named_entities,bralette_named_entities,brandy_named_entities,brandymelville_named_entities,brat_named_entities,bratz_named_entities,britney_named_entities,broderie_named_entities,brown_named_entities,bts_named_entities,bubblegum_named_entities,burberry_named_entities,burgundy_named_entities,bustier_named_entities,busty_named_entities,california_named_entities,calvin_named_entities,cami_named_entities,camisole_named_entities,camo_named_entities,canada_named_entities,cara_named_entities,carhartt_named_entities,carla_named_entities,carlson_named_entities,caroline_named_entities,cb_named_entities,cdg_named_entities,chamberlain_named_entities,chanel_named_entities,cherry_named_entities,cherub_named_entities,chinese_named_entities,choker_named_entities,christy_named_entities,clark_named_entities,clasp_named_entities,club_named_entities,clueless_named_entities,co_named_entities,comfy_named_entities,coord_named_entities,corona_named_entities,corset_named_entities,cosplay_named_entities,cottagecore_named_entities,couture_named_entities,cowgirl_named_entities,crewneck_named_entities,croptop_named_entities,cutie_named_entities,cyber_named_entities,daisy_named_entities,davidson_named_entities,day_named_entities,de_named_entities,deadstock_named_entities,decathlon_named_entities,delia_named_entities,depop_named_entities,depopfamous_named_entities,depopseller_named_entities,depopusa_named_entities,diamante_named_entities,diamond_named_entities,dickie_named_entities,dior_named_entities,disco_named_entities,disney_named_entities,ditsy_named_entities,diy_named_entities,dm_named_entities,doc_named_entities,dolce_named_entities,dollskill_named_entities,dot_named_entities,dr_named_entities,dragon_named_entities,drmarten_named_entities,dupe_named_entities,dutch_named_entities,dye_named_entities,eagle_named_entities,earre_named_entities,earring_named_entities,eboy_named_entities,egirl_named_entities,eilish_named_entities,elasticate_named_entities,elizabeth_named_entities,ellish_named_entities,embroider_named_entities,emma_named_entities,emo_named_entities,erica_named_entities,evening_named_entities,express_named_entities,fall_named_entities,faux_named_entities,fendi_named_entities,fenty_named_entities,fila_named_entities,fishnet_named_entities,flea_named_entities,fleece_named_entities,flipflop_named_entities,flowery_named_entities,flowy_named_entities,force_named_entities,frankie_nam

## Get data ready for modelling

In [9]:
df = df.drop(
    ["description", "description_truecase", "description_nlp", "named_entities"], axis=1
)
df[:1]

,product_id,label,proportion_of_stopwords,average_length_of_word,proportion_of_numbers,abercrombie_named_entities,adida_named_entities,adidas_named_entities,air_named_entities,airmax_named_entities,aj_named_entities,aje_named_entities,alexander_named_entities,alice_named_entities,alt_named_entities,amara_named_entities,american_named_entities,angel_named_entities,angele_named_entities,anime_named_entities,approx_named_entities,argyle_named_entities,ariana_named_entities,ashley_named_entities,ashlyn_named_entities,asos_named_entities,australia_named_entities,autumn_named_entities,babe_named_entities,babydoll_named_entities,babygirl_named_entities,backless_named_entities,baddie_named_entities,baguette_named_entities,baker_named_entities,balenciaga_named_entities,bandana_named_entities,bandeau_named_entities,bape_named_entities,barbie_named_entities,bardot_named_entities,barkingmadz_named_entities,bay_named_entities,bdg_named_entities,beach_named_entities,bead_named_entities,beanie_named_entities,bebe_named_entities,bec_named_entities,bella_named_entities,belle_named_entities,bershka_named_entities,betty_named_entities,bikini_named_entities,billie_named_entities,black_named_entities,blair_named_entities,bleach_named_entities,blogger_named_entities,blue_named_entities,bodycon_named_entities,bodysuit_named_entities,bohemian_named_entities,boho_named_entities,boob_named_entities,boohoo_named_entities,boop_named_entities,boujee_named_entities,boyfriend_named_entities,bra_named_entities,braid_named_entities,bralet_named_entities,bralette_named_entities,brandy_named_entities,brandymelville_named_entities,brat_named_entities,bratz_named_entities,britney_named_entities,broderie_named_entities,brown_named_entities,bts_named_entities,bubblegum_named_entities,burberry_named_entities,burgundy_named_entities,bustier_named_entities,busty_named_entities,california_named_entities,calvin_named_entities,cami_named_entities,camisole_named_entities,camo_named_entities,canada_named_entities,cara_named_entities,carhartt_named_entities,carla_named_entities,carlson_named_entities,caroline_named_entities,cb_named_entities,cdg_named_entities,chamberlain_named_entities,chanel_named_entities,cherry_named_entities,cherub_named_entities,chinese_named_entities,choker_named_entities,christy_named_entities,clark_named_entities,clasp_named_entities,club_named_entities,clueless_named_entities,co_named_entities,comfy_named_entities,coord_named_entities,corona_named_entities,corset_named_entities,cosplay_named_entities,cottagecore_named_entities,couture_named_entities,cowgirl_named_entities,crewneck_named_entities,croptop_named_entities,cutie_named_entities,cyber_named_entities,daisy_named_entities,davidson_named_entities,day_named_entities,de_named_entities,deadstock_named_entities,decathlon_named_entities,delia_named_entities,depop_named_entities,depopfamous_named_entities,depopseller_named_entities,depopusa_named_entities,diamante_named_entities,diamond_named_entities,dickie_named_entities,dior_named_entities,disco_named_entities,disney_named_entities,ditsy_named_entities,diy_named_entities,dm_named_entities,doc_named_entities,dolce_named_entities,dollskill_named_entities,dot_named_entities,dr_named_entities,dragon_named_entities,drmarten_named_entities,dupe_named_entities,dutch_named_entities,dye_named_entities,eagle_named_entities,earre_named_entities,earring_named_entities,eboy_named_entities,egirl_named_entities,eilish_named_entities,elasticate_named_entities,elizabeth_named_entities,ellish_named_entities,embroider_named_entities,emma_named_entities,emo_named_entities,erica_named_entities,evening_named_entities,express_named_entities,fall_named_entities,faux_named_entities,fendi_named_entities,fenty_named_entities,fila_named_entities,fishnet_named_entities,flea_named_entities,fleece_named_entities,flipflop_named_entities,flowery_named_entities,flowy_named_entities,force_named_entities,frankie_named_entities,french_named_entities,funky_named_entities,gaga_name

## Split into train and test

In [10]:
X_train = df.drop("label", axis=1)
y_train = df["label"]
X_test = df[df["product_id"].isin(list(test["product_id"]))].drop("label", axis=1)
y_test = df[df["product_id"].isin(list(test["product_id"]))]["label"]

## Fit model

In [11]:
param_dist = {"objective": "binary:logistic", "n_estimators": 2}

clf = xgb.XGBClassifier(**param_dist)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric="logloss",
    verbose=True,
)

[0]	validation_0-logloss:0.51540	validation_1-logloss:0.50234
[1]	validation_0-logloss:0.40579	validation_1-logloss:0.38368


/Users/alicejacques/.virtualenvs/ml-coding-test/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=2, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Predict on model

In [12]:
y_pred = clf.predict(X_test)

## Evaluate

In [13]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {(accuracy * 100.0):.2f}%")

Accuracy: 95.20%


## Oh my goodness, I'm a brilliant Data Scientist

## Check features

In [14]:
features = pd.DataFrame(
    zip(X_train.columns, clf.feature_importances_),
    columns=["feature_name", "feature_importance"],
)
features.sort_values("feature_importance", ascending=False)

,feature_name,feature_importance
981,vintage_description,0.149434
1000,yk_description,0.111777
847,polly_description,0.064823
2,average_length_of_word,0.052463
808,nike_description,0.052402
838,pink_description,0.043354
911,size_description,0.042406
3,proportion_of_numbers,0.038070
939,summer_description,0.037827
861,punk_description,0.036967
